In [1]:
import numpy as np
from recstate2mode import UW_,UV_,C_,mu_,Sigma_,G_matrix

In [2]:
gamma = np.array([0,0],dtype = np.complex128)
phi = np.array([0,0],dtype = np.complex128)
zeta = np.array([0,0],dtype = np.complex128)
theta = 0
psi = 0
theta1 = 0.3
psi1 = 0.5
cutoff_dim = 4
cutoff= 4
cutoff_plus = 10


In [3]:
old_state = np.random.rand(cutoff_dim,cutoff_dim) + 1.0j*np.random.rand(cutoff_dim,cutoff_dim)
old_state  /= np.linalg.norm(old_state)

In [4]:
old_state

array([[0.08282601+0.10804998j, 0.18455925+0.0262027j ,
        0.26508316+0.00403834j, 0.25363293+0.17037411j],
       [0.08897329+0.24856917j, 0.1145089 +0.17107191j,
        0.05209445+0.06727625j, 0.15844659+0.2377667j ],
       [0.25813197+0.30247816j, 0.04237917+0.04715975j,
        0.2552329 +0.15636546j, 0.25428961+0.16713571j],
       [0.12234709+0.10323259j, 0.02382239+0.17178233j,
        0.1590269 +0.28462191j, 0.20531063+0.18336882j]])

In [5]:
W = UW_(phi,theta1,psi1)
V = UV_(theta,psi)
    
C = C_(gamma, W, zeta, V)
mu = mu_(gamma, W, zeta, V)
Sigma = Sigma_(W, zeta, V)

cutoff = old_state.shape[0]
dtype = old_state.dtype


sqrt = np.sqrt(np.arange(cutoff+1, dtype=dtype))

G_00pq = np.zeros((cutoff, cutoff), dtype=dtype)


G_00pq[0,0] = C
for q in range(1, cutoff):
    G_00pq[0,q] = (mu[3]*G_00pq[0,q-1] - Sigma[3,3]*sqrt[q-1]*G_00pq[0,q-2])/sqrt[q]


for p in range(1,cutoff):
    for q in range(0,cutoff):
        G_00pq[p,q] = (mu[2]*G_00pq[p-1,q] - Sigma[2,2]*sqrt[p-1]*G_00pq[p-2,q] - Sigma[2,3]*sqrt[q]*G_00pq[p-1,q-1])/sqrt[p]

In [6]:
mu

array([ 0.+0.j,  0.+0.j, -0.-0.j, -0.-0.j])

In [7]:
Sigma

array([[ 0.        +0.j        ,  0.        +0.j        ,
        -0.95533649+0.j        ,  0.25934338-0.14167993j],
       [ 0.        +0.j        ,  0.        +0.j        ,
        -0.25934338-0.14167993j, -0.95533649+0.j        ],
       [-0.95533649+0.j        , -0.25934338-0.14167993j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.25934338-0.14167993j, -0.95533649+0.j        ,
         0.        +0.j        ,  0.        +0.j        ]])

In [8]:
G_00pq

array([[1.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j],
       [0.+0.j, 0.-0.j, 0.+0.j, 0.-0.j],
       [0.-0.j, 0.+0.j, 0.-0.j, 0.+0.j]])

In [9]:
G = G_matrix(gamma, phi, theta1, psi1, zeta, theta, psi,cutoff)

In [10]:
# R_00^jk = a_dagger^j \G_00pq> b^k  * |old_state>

R = np.zeros((cutoff, cutoff, cutoff+1, cutoff+1), dtype=dtype)
sqrts = np.zeros((cutoff, cutoff), dtype=dtype)
for m in range(cutoff):
    for n in range(cutoff):
        sqrts[m,n]= np.sqrt(m+1)*np.sqrt(n+1)
        
for j in range(cutoff):
    for k in range(cutoff):
        if j==0 and k ==0:
            R[0,0,j,k] = np.sum(G_00pq*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G_00pq[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[:,k:] = G_00pq[:,:cutoff-k] * sqrt[1:cutoff-k+1]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,k:] = G_00pq[:cutoff-j,:cutoff-k] * sqrts[:cutoff-j,:cutoff-k]
            R[0,0,j,k] = np.sum(G_00pq2*old_state)

#R00jk is correct
    
#R_0n^jk
for n in range(1,cutoff):
    for k in range(0,cutoff): 
        for j in range(0,cutoff):
         
            R[0,n,j,k] = mu[1]/sqrt[n]*R[0,n-1,j,k] - Sigma[1,1]/sqrt[n]*sqrt[n-1]*R[0,n-2,j,k] - Sigma[1,2]/sqrt[n]*R[0,n-1,j+1,k] - Sigma[1,3]/sqrt[n]*R[0,n-1,j,k+1]
#R0100 correct but others are not correct
print(R[0,1,:cutoff,:cutoff])

for m in range(1,cutoff):
    for n in range(0,cutoff):
        for j in range(0,cutoff):
            for k in range(0,cutoff):
                R[m,n,j,k] = mu[0]/sqrt[m]*R[m-1,n,j,k] - Sigma[0,0]/sqrt[m]*sqrt[m-1]*R[m-2,n,j,k] - Sigma[0,1]*sqrt[n]/sqrt[m]*R[m-1,n-1,j,k] - Sigma[0,2]/sqrt[m]*R[m-1,n,j+1,k] - Sigma[0,3]/sqrt[m]*R[m-1,n,j,k+1]


[[0.16417355+0.1021029j  0.25870328+0.06444796j 0.24628345+0.187593j
  0.0074053 +0.08411192j]
 [0.13348426+0.27844907j 0.0540769 +0.08250631j 0.19540893+0.30386093j
  0.04226855+0.07937328j]
 [0.05759029+0.08916025j 0.22567338+0.19730739j 0.24384951+0.25601657j
  0.02726627+0.07664388j]
 [0.0227584 +0.16410993j 0.1519242 +0.27190969j 0.19614074+0.17517892j
  0.        +0.j        ]]


In [11]:
#example for R_01^01
-Sigma[1,2]*R[0,0,1,1]/sqrt[1]-Sigma[1,3]*R[0,0,0,2]/sqrt[1]

(0.2587032782610768+0.06444795937104594j)

In [12]:
#example for R_01^01
j=0
k=1
G22 = G[0,1,:,:]
a = np.zeros((cutoff,cutoff),dtype=dtype)
if j==0 and k ==0:
    a[j,k] = np.sum(G22*old_state)

elif j>0 and k==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,:] = G22[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
    a[j,k]  = np.sum(G_00pq2*old_state)

elif k>0 and j==0:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[:,k:] = G22[:,:cutoff-k] * sqrt[1:cutoff-k+1]
    a[j,k]  = np.sum(G_00pq2*old_state)
else:
    G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
    G_00pq2[j:,k:] = G22[:cutoff-j,:cutoff-k]  * sqrts[:cutoff-j,:cutoff-k]
    a[j,k]  = np.sum(G_00pq2*old_state)
print(a[j,k])


(0.3636002168769792+0.06604598617563395j)


In [13]:
#Use g_matrix to get the R matrix
m=0
n=1
G22 = G[m,n,:,:]
a = np.zeros((cutoff,cutoff),dtype=dtype)
for j in range(cutoff):
    for k in range(cutoff):
        if j==0 and k ==0:
            a[j,k] = np.sum(G22*old_state)

        elif j>0 and k==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,:] = G22[:cutoff-j,:] * sqrt[1:cutoff-j+1,None]
            a[j,k]  = np.sum(G_00pq2*old_state)

        elif k>0 and j==0:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[:,k:] = G22[:,:cutoff-k] * sqrt[1:cutoff-k+1]
            a[j,k]  = np.sum(G_00pq2*old_state)
        else:
            G_00pq2 = np.zeros((cutoff, cutoff), dtype=dtype)
            G_00pq2[j:,k:] = G22[:cutoff-j,:cutoff-k]  * sqrts[:cutoff-j,:cutoff-k]
            a[j,k]  = np.sum(G_00pq2*old_state)
print("R_01",a)# a = R[m,n,:,:]

R_01 [[0.16417355+0.1021029j  0.36360022+0.06604599j 0.34664938+0.2550123j
  0.0074053 +0.08411192j]
 [0.14346255+0.32609102j 0.07647628+0.11668154j 0.27634995+0.42972425j
  0.05977676+0.11225076j]
 [0.06467497+0.10742989j 0.31915035+0.27903479j 0.34485528+0.36206211j
  0.03856033+0.10839082j]
 [0.0227584 +0.16410993j 0.21485326+0.38453837j 0.27738489+0.24774041j
  0.        +0.j        ]]


In [14]:
R[0,1,:cutoff,:cutoff]

array([[0.16417355+0.1021029j , 0.25870328+0.06444796j,
        0.24628345+0.187593j  , 0.0074053 +0.08411192j],
       [0.13348426+0.27844907j, 0.0540769 +0.08250631j,
        0.19540893+0.30386093j, 0.04226855+0.07937328j],
       [0.05759029+0.08916025j, 0.22567338+0.19730739j,
        0.24384951+0.25601657j, 0.02726627+0.07664388j],
       [0.0227584 +0.16410993j, 0.1519242 +0.27190969j,
        0.19614074+0.17517892j, 0.        +0.j        ]])

In [15]:
#Result from recursive process
R[:,:,0,0]

array([[ 0.08282601+0.10804998j,  0.16417355+0.1021029j ,
         0.17134349+0.10797199j,  0.09201985+0.09600846j],
       [ 0.03342281+0.25682004j,  0.05129842+0.28595147j,
        -0.02798599+0.05998924j,  0.06849665+0.14239007j],
       [ 0.10277254+0.14354319j,  0.02039057+0.03244342j,
        -0.00669351+0.04766186j,  0.02701845+0.0738171j ],
       [ 0.03047712+0.03156004j, -0.05254075+0.0369466j ,
        -0.03591777+0.0253554j ,  0.00573157+0.00511903j]])

In [16]:
#Result from G matrix
np.einsum("cd,abcd->ab",old_state,G)

array([[ 0.08282601+0.10804998j,  0.16417355+0.1021029j ,
         0.23926094+0.1187879j ,  0.22257325+0.19997196j],
       [ 0.03342281+0.25682004j,  0.03340031+0.34591291j,
        -0.1009213 +0.08901967j,  0.14866611+0.31548446j],
       [ 0.17552536+0.21875007j,  0.05739005+0.06793765j,
        -0.04632552+0.09841107j,  0.09359466+0.25570994j],
       [ 0.08613786+0.08530314j, -0.09034998+0.09240627j,
        -0.12442281+0.08783367j,  0.03438944+0.03071419j]])

In [17]:
np.sum(G[0,0,:,:]*old_state)

(0.08282601003517769+0.10804997686262965j)

In [18]:
np.sum(G[0,1,:,:]*old_state)

(0.16417355377621168+0.1021028968907539j)